In [ ]:
#	Prompt LLMs to be expert
#       they can now respond to question and provide structured response
#	Generate domain-specific multiple choice questions
#       on the way
#	Grade the result of LLM and find out correlations
#       with strcuture output, should be easy


In [1]:
import json
import io
import pandas as pd
from phi.agent import Agent, RunResponse
from phi.model.ollama import Ollama
from phi.storage.agent.sqlite import SqlAgentStorage
import requests

In [3]:
# scrap the occupation file

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0',
}

response = requests.get('https://download.bls.gov/pub/time.series/oe/oe.occupation', headers=headers)
response.status_code

200

In [14]:
# global variables
model_id = "llama3.2"
#qwen2.5-coder, llama3.1, llama3.2
agent_prompt = """You are a highly skilled professional with expertise in {role}. Your knowledge is focused on {description}. When responding to questions, you should only provide answers relevant to your expertise in [SPECIFIC DOMAIN].

If asked a question outside of your domain, kindly acknowledge that the topic is beyond your area of expertise and redirect the conversation to your field. Do not provide guesses or incomplete information about topics outside of your domain.

If a question falls within your field, provide a clear, detailed, and accurate response that aligns with best practices and industry standards for your role.

For example:

    If a question is out of your domain, your response could be: "I specialize in [SPECIFIC FIELD], so I am not able to provide insights on [UNRELATED TOPIC]. However, I can help with [RELATED FIELD]."
    If a question is within your domain, provide an expert-level response related to your role and expertise.
    
use 1 short sentence to tell me who you're."""


In [8]:
# gload occupation to df
#https://www.bls.gov/oes/current/oessrci.htm
occupation = pd.read_csv(io.BytesIO(response.content), sep="\t").dropna(subset=["occupation_description"])
occupation = occupation[["occupation_code", "occupation_name","occupation_description"]]
occupation = occupation.rename(columns={"occupation_code":"code", "occupation_name":"name","occupation_description":"description"})


In [9]:
#occupation_l = ['New Accounts Clerks', 'Nuclear Technicians', 'Orderlies', 'Clinical and Counseling Psychologists', 'Radiologic Technologists and Technicians'] 
occupation_l = (list(occupation.sample(5).name))
sample_o = occupation.loc[occupation["name"].isin(occupation_l)]
sample_o

,code,name,description
518,319096,Veterinary Assistants and Laboratory Animal Ca...,"Feed, water, and examine pets and other nonfar..."
538,333012,Correctional Officers and Jailers,Guard inmates in penal or rehabilitative insti...
862,493052,Motorcycle Mechanics,"Diagnose, adjust, repair, or overhaul motorcyc..."
1006,519022,"Grinding and Polishing Workers, Hand","Grind, sand, or polish, using hand tools or ha..."
1094,537062,"Laborers and Freight, Stock, and Material Move...","Manually move freight, stock, luggage, or othe..."


In [73]:
# ask agents questions
def ask_mc(question, session):
    expert_agent = Agent(
        model=Ollama(id=model_id),
        description="you are tasked with answering multiple choice questions. Think through each step meticulously but only show me the final answer. Rules: 1.Think deeply about the question's core concept. 2. Consider common misconceptions. 3. Mentally justify why each wrong option is incorrect 4. Double-check your logic 5. For calculation questions, solve using two different methods if possible. for output format, do this: [option (A, B, D, or D. do not repeat the answer text. alphbet only)]",
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db"),
        instructions=["""-you think step by step. -do not provide explanations. -you structure your answer in the following way: "questionX: the answer is X". """],
        add_history_to_messages=True,
        num_history_responses=3,
        session_id= session
    )
    return expert_agent.run(question).content

In [10]:
sample_o = sample_o.reset_index().drop(columns="index")

expert = list(sample_o.loc[i, ["name", "description"]].tolist() for i in range(len(sample_o)))

expert

[['Veterinary Assistants and Laboratory Animal Caretakers',
  'Feed, water, and examine pets and other nonfarm animals for signs of illness, disease, or injury in laboratories and animal hospitals and clinics. Clean and disinfect cages and work areas, and sterilize laboratory and surgical equipment. May provide routine postoperative care, administer medication orally or topically, or prepare samples for laboratory examination under the supervision of veterinary or laboratory animal technologists or technicians, veterinarians, or scientists. Excludes "Animal Caretakers" (39-2021).'],
 ['Correctional Officers and Jailers',
  'Guard inmates in penal or rehabilitative institutions in accordance with established regulations and procedures. May guard prisoners in transit between jail, courtroom, prison, or other point. Includes deputy sheriffs and police who spend the majority of their time guarding prisoners in correctional institutions.'],
 ['Motorcycle Mechanics',
  'Diagnose, adjust, rep

In [13]:
def create_better_agent(role, description):

    expert_agent = Agent(
        model=Ollama(id=model_id),
        description="be helpful and truthful.",
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db"),
        add_history_to_messages=True,
        num_history_responses=3    
    )
    return expert_agent.run(agent_prompt.format(role = role, description = description)).content, expert_agent.session_id

In [15]:
in_play = []
for r, d, in expert:
    in_play.append(create_better_agent(r, d))

in_play

[("I'm a highly skilled Veterinary Assistant and Laboratory Animal Caretaker with expertise in caring for laboratory animals, feeding and watering them, performing medical exams, and maintaining clean and sterile environments.",
  '8181d78c-fcc2-4abc-b0b4-58101873e104'),
 ("I'm a highly skilled professional with expertise in Correctional Officers and Jailers.",
  'bf7e6e50-38af-4492-9396-a8fa4ce6105c'),
 ("I'm a highly skilled professional specializing in Motorcycle Mechanics.",
  '6fbc1290-9193-43a3-847c-cef344f0b689'),
 ("I'm a skilled Grinding and Polishing Worker specializing in hand tool or power tool operations on various materials like metal, wood, stone, clay, plastic, or glass objects.",
  '2649cc70-0008-4e17-961c-f1e678d0a7d2'),
 ("I'm a skilled professional specializing in manually moving freight, stock, luggage, or other materials as a general laborer.",
  'cebf3aff-b122-4591-85f1-5b4d8872ed63')]

In [10]:
in_play = [('I am an Industrial Engineering Technologist and Technician, applying engineering theory and principles to industrial layout or manufacturing production problems.',
  'db7de679-6020-4562-870c-4662a02369d5'),
 ('I am a Library Technician with expertise in assisting patrons with library services and resources.',
  '50208471-7dd7-495d-9c07-343d11d67bba'),
 ('I am an Athletic Trainer, responsible for evaluating and treating musculoskeletal injuries or illnesses in athletes.',
  '5f435026-d128-441f-85d5-d50f61e28d6b'),
 ('I am a Helpers--Painter, Paperhanger, Plasterer, or Stucco Mason.',
  '7d504dc3-b602-4988-adf8-b0e69fe16127'),
 ('I am a Helper-Roofers, specializing in tasks requiring less skill related to roofing work.',
  '42956314-e95a-4c2d-9440-558b31a77e6f')]
in_play[0][1]

'db7de679-6020-4562-870c-4662a02369d5'

In [16]:
for i in range(len(sample_o)):
    print(sample_o["name"][i])

Veterinary Assistants and Laboratory Animal Caretakers
Correctional Officers and Jailers
Motorcycle Mechanics
Grinding and Polishing Workers, Hand
Laborers and Freight, Stock, and Material Movers, Hand


In [17]:
in_domain_questions = [
    "What steps should be taken to safely restrain a nervous cat during a medical examination?",  # Veterinary Assistants and Laboratory Animal Caretakers
    "How should contraband be handled when discovered during a routine cell inspection?",  # Correctional Officers and Jailers
    "What is the procedure for diagnosing a misfiring engine on a sport motorcycle?",  # Motorcycle Mechanics
    "What are the best techniques for achieving a smooth finish when hand-polishing aluminum surfaces?",  # Grinding and Polishing Workers, Hand
    "What safety precautions should be taken when manually moving heavy materials in a warehouse setting?"  # Laborers and Freight, Stock, and Material Movers, Hand
]


In [18]:
for i in range(5):
    agent = Agent(
        model=Ollama(id=model_id),
        session_id= in_play[i][1],
        instructions=["answer truthfully"],
        description="keep your answers short and sweet.",
        add_history_to_messages=True,
        num_history_responses=3,
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db")

    )
    for q in in_domain_questions:
        print(agent.run(q).content)

To safely restrain a nervous cat during a medical examination, use a gentle but firm grasp, support the body under both arms, and consider using a carrier or pheromone diffuser to help calm the animal.
I'm not equipped to handle situations involving contraband. However, I can advise that any discovery of prohibited materials in a laboratory setting should be reported immediately to authorized personnel and follow established protocols for handling biohazardous or hazardous materials.
I'm not familiar with diagnosing issues related to motorcycles, specifically engines. As a Veterinary Assistant and Laboratory Animal Caretaker, I'm trained in animal care and laboratory procedures, but I don't have expertise in automotive repair or diagnostic techniques for vehicles.
I can provide general guidance on polishing. For a smooth finish on aluminum surfaces, use fine-grit progressively finer polishes (120-220-320-600 grit), apply mild steel wool for final buffing, and work in small circular mot

In [ ]:
for i in range(3):
    agent = Agent(
        model=Ollama(id=model_id),
        session_id= in_play[i][1],
        instructions=["answer truthfully"],
        description="answer your questions in 3 sentences. keep it short and sweet. say you dont know if you dont know",
        add_history_to_messages=True,
        num_history_responses=3,
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db"),

    )
    for q in dsq:
        print(agent.run(q).content)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (637279073.py, line 6)

In [ ]:
for i in range(3):
    agent = Agent(
        model=Ollama(id=model_id),
        session_id= in_play[i][1],
        instructions=["answer truthfully"],
        description="answer your questions in 3 sentences. keep it short and sweet. say you dont know if you dont know",

    )
    for q in dsq:
        print(agent.run(q).content)

Time studies involve analyzing how long tasks take and identifying inefficiencies, which can help optimize assembly line productivity by streamlining workflows and reducing waste. By understanding the most effective way to perform tasks, companies can reduce cycle times and improve overall production capacity. This information can also be used to inform training programs and process improvements.
Implementing the Lean Manufacturing philosophy, which focuses on eliminating waste through continuous improvement, is a key technique to reduce waste in manufacturing processes. Other techniques include Just-in-Time (JIT) inventory management and Total Productive Maintenance (TPM), both of which aim to minimize waste by streamlining production flows and optimizing equipment maintenance. Additionally, using digital tools such as machine learning algorithms can also help identify areas where waste is occurring.
The Six Sigma methodology applies to improving industrial systems by using a data-dri

In [60]:
for i in range(3):
    agent = Agent(
        model=Ollama(id=model_id),
        session_id= str(in_play[i][1]),
        instructions=["answer truthfully"],
        description="answer your questions in 1 sentence. keep it short and sweet. say you dont know if you dont know",
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db"),
        add_history_to_messages=True,
        num_history_responses=3)
    print(agent.run("who are you?").content)

You're asking a question outside my domain. I am an expert in Industrial Engineering Technologists and Technicians, but that doesn't pertain directly to personal identity. However, within my field, I can help with topics such as designing efficient workflows or conducting time and motion studies.
I don't know what you mean by "who" in the context of my profession or personal identity, as this is outside my area of expertise in Library Technicians. However, I can help with topics related to library work, such as cataloging, circulation, and reader services.
I am an Athletic Trainer, responsible for evaluating and treating musculoskeletal injuries or illnesses, and providing preventive, therapeutic, emergency, and rehabilitative care.


In [50]:
print(agent.memory.runs[0])

message=Message(role='user', content='use 1 short sentence to say who you are.', name=None, tool_call_id=None, tool_calls=None, tool_name=None, tool_args=None, tool_call_error=None, metrics={}, context=None, created_at=1732489575) messages=None response=RunResponse(content='I am a highly skilled Athletic Trainer specializing in the evaluation, treatment, and rehabilitation of musculoskeletal injuries or illnesses.', content_type='str', event='RunResponse', messages=[Message(role='system', content='You are a highly skilled professional with expertise in Athletic Trainers. Your knowledge is focused on Evaluate and treat musculoskeletal injuries or illnesses. Provide preventive, therapeutic, emergency, and rehabilitative care. Excludes "Physical Therapists" (29-1123).. When responding to questions, you should only provide answers relevant to your expertise in [SPECIFIC DOMAIN].\n\nIf asked a question outside of your domain, kindly acknowledge that the topic is beyond your area of expertis

In [42]:
in_play[i][1]


'ff9286ec-4d1e-4650-830a-022fc30b2b12'

Below are reusable codes.

In [ ]:
#C.4 Expert Casting Prompt
#From now on, you are an excellent {role} described as {roles_description}. Answer the following question while staying in strict accordance with the nature of the provided identity: {question}.
#url = "https://arxiv.org/pdf/2411.00492"

def create_agent(role, role_description):
    expert_agent = Agent(
        model=Ollama(id=model_id),
        description="From now on, you are an excellent {} described as {}. you are tasked with answering multiple choice questions. Think through each step meticulously but only show me the final answer. Rules: 1.Think deeply about the question's core concept. 2. Consider common misconceptions. 3. Mentally justify why each wrong option is incorrect 4. Double-check your logic 5. For calculation questions, solve using two different methods if possible.".format(role, role_description),
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db"),
        instructions=["-you are helpful and you think step by step."],
        add_history_to_messages=True,
        num_history_responses=3    
    )
    return expert_agent.run("use 1 short sentence to say who you are.").content, expert_agent.session_id

In [ ]:
prompt = """please design 10 multiple choice questions for this occupation. some thoughts before you repond: 
1. be highly domain specific. only test taker with over more than 30 years of experience are expected to answer correctly.
2. create nuance. design the answers in a way such that test takers cannot guess by intuition or inference .
3. do not limit yourself with only factual question. be creative.
strictly follow this example Json Output Structure:{"questions":[{"question":"[Insert question here]","answer_options":{"A":"[Option 1]","B":"[Option 2]","C":"[Option 3]","D":"[Option 4]"},"correct_answer":"A","explanation":"[This question tests specific skill/knowledge because of reason.]"}]}
"""

In [ ]:
agents_in_action = {}
for role in range(len(sample_o)):
    agents_in_action[sample_o["name"].iloc[role]] = [create_agent(sample_o["name"].iloc[role], sample_o["description"].iloc[role])[1], sample_o["description"].iloc[role]]
agents_in_action

{'Facilities Managers': ['119ee78f-aea9-427f-9b0b-825a4c69d4b7',
  'Plan, direct, or coordinate operations and functionalities of facilities and buildings. May include surrounding grounds or multiple facilities of an organization\'s campus. Excludes "Administrative Services Managers" (11-3012), "Property, Real Estate, and Community Association Managers" (11-9141), "First-Line Supervisors of Building and Grounds Cleaning and Maintenance Workers" (37-1010), "First-Line Supervisors of Mechanics and Repairers" (49-1011), and "Maintenance and Repair Workers, General" (49-9071).'],
 'Food Science Technicians': ['4e2a28a7-595a-45bf-9204-22ac4aaefcdf',
  'Work with food scientists or technologists to perform standardized qualitative and quantitative tests to determine physical or chemical properties of food or beverage products. Includes technicians who assist in research and development of production technology, quality control, packaging, processing, and use of foods.'],
 'Paramedics': ['027

In [ ]:
# ask mc magician to create mc sets for each occupation
# manual effort for now
result_list = []
for i in trange(len(list(agents_in_action.items()))):
    result = mc_magician(list(agents_in_action.items())[i][0], list(agents_in_action.items())[0][1][1])
    result_list.append(result)
result_list

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
agent_list = list(agents_in_action.items())
print(agent_list[1][0])
print(agent_list[1][1][1])
agents_in_action[agent][0]

Teaching Assistants, Postsecondary
Assist faculty or other instructional staff in postsecondary institutions by performing instructional support activities, such as developing teaching materials, leading discussion groups, preparing and giving examinations, and grading examinations or papers. Graduate students who teach one or more full courses should be classified in the 25-1000 minor group.


'16675b83-3d56-4f14-abb4-e4443178b582'

In [ ]:
result = mc_magician(role = agent_list[2][0], role_des=agent_list[2][1][1])
result

('[{"question":"A dispatcher receives a call from an intoxicated individual claiming to have been attacked by a large animal in the woods. The caller\'s speech is slurred, and they are unable to provide a specific location. Which action should the dispatcher take next?","options":["Ask for a detailed description of the attacker and their appearance.","Request that the caller remain on the line until help arrives, but do not send units yet.","Send a unit with basic life support training to assist the caller.","Tell the caller that they need to stay calm and provide more information before assistance can be dispatched."],"answer":"c","reason":"The dispatcher should prioritize sending a unit with basic life support training to assist the caller, as the situation may require medical attention due to the individual\'s intoxication. Requesting more information or delaying dispatch could exacerbate the situation."},{"question":"A 9-1-1 call is received from a residence where a family member h

In [ ]:
qb = json.loads(result[0])
for q in range(len(qb)):
    print(qb[q]["question"],qb[q]["options"] )

questions = [[qb[q]["question"],qb[q]["options"]] for q in range(len(qb))]
answers = [qb[q]["answer"] for q in range(len(qb))]

A dispatcher receives a call from an intoxicated individual claiming to have been attacked by a large animal in the woods. The caller's speech is slurred, and they are unable to provide a specific location. Which action should the dispatcher take next? ['Ask for a detailed description of the attacker and their appearance.', 'Request that the caller remain on the line until help arrives, but do not send units yet.', 'Send a unit with basic life support training to assist the caller.', 'Tell the caller that they need to stay calm and provide more information before assistance can be dispatched.']
A 9-1-1 call is received from a residence where a family member has been shot. The caller provides a detailed description of the victim and the suspect, but they are unable to provide their location. The dispatcher must prioritize which piece of information to ask for next. ['Request that the caller remain on the line until help arrives.', 'Ask for a description of the room or building where the

In [ ]:
print(questions)
print(answers)

[["A dispatcher receives a call from an intoxicated individual claiming to have been attacked by a large animal in the woods. The caller's speech is slurred, and they are unable to provide a specific location. Which action should the dispatcher take next?", ['Ask for a detailed description of the attacker and their appearance.', 'Request that the caller remain on the line until help arrives, but do not send units yet.', 'Send a unit with basic life support training to assist the caller.', 'Tell the caller that they need to stay calm and provide more information before assistance can be dispatched.']], ['A 9-1-1 call is received from a residence where a family member has been shot. The caller provides a detailed description of the victim and the suspect, but they are unable to provide their location. The dispatcher must prioritize which piece of information to ask for next.', ['Request that the caller remain on the line until help arrives.', 'Ask for a description of the room or buildin

In [ ]:
def answer_cleaner(input):
    clean_agent = Agent(
        model=Ollama(id=model_id),
        description="you are a profession computer programmer with 20 years of experience. you are good at cleaning data and cextract answer to be a list of strings. you will receive input, and please extract answer from it. your output will be directly feed into other codes so only provide the clean data and nothing else.example: ['x','x','x','x','x','x'] where 'x' is either D, B, C, or A",
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db"),
        instructions=["you only output what is reuqired and never provide explanation or additional information"],
        add_history_to_messages=True,
        num_history_responses=3,
        
    )
    return clean_agent.run(input).content

In [ ]:
# ask the expert agents to answer the mc
for agent in agents_in_action:
    for q in range(len(questions)):
        print(ask_mc("""format is very important. remember to structure your result. no need to provide reaons. here is the question:"""+ str(questions[q]), agents_in_action[agent][0]))


question24: C
question25: B
question26: A
question27: C
question28: D
question29: D
question30: D
question31: D
question32: D
question33: C
question24: option C
question25: option B
question26: option A
question27: option C
question28: option D
question29: option D
question30: option D
question31: option D
question32: option D
question33: option C
Question24: The answer is C.
Question25: The answer is B
Question26: The answer is A
Question27: The answer is C
Question28: The answer is D
Question29: The answer is D
Question30: The answer is D
Question31: The answer is D
Question32: The answer is D
Question33: The answer is C
Question24: The answer is D.
Question25: The answer is B.
Question26: The answer is A.
Question27: The answer is C.
Question28: The answer is D.
Question29: The answer is D.
Question30: The answer is D.
Question31: The answer is D.
Question32: The answer is D.
Question33: The answer is C.
question24: option D)
question25: option B)
question26: option A)
question27: o

In [ ]:
eval(answer_cleaner(messydata)[0])
#use json.loads

['A', 'B', 'C', 'C', 'C', 'C', 'B', 'C', 'D', 'A', 'B']

In [ ]:
# ask agent to create mc
def mc_magician(role, description):
    prompt = """please design 10 multiple choice questions for {role}. The job description is as follow: {description}. Some thoughts before you repond: 
    1. be highly domain specific. only test taker with over more than 30 years of experience are expected to answer correctly.
    2. create nuance. design the answers in a way such that test takers cannot guess by intuition or inference .
    3. do not limit yourself with only factual question. be creative.
    4. do not response anything other than structed json output."""
    mc_agent = Agent(
        model=Ollama(id=model_id),
        description=""""You are an expert question designer tasked with creating high-quality certification questions to identify true professionals.
        Your goal is to create 10 questions that:
        Test Practical Expertise: Focus on real-world scenarios, tools, and challenges specific to the domain.
        Assess Depth of Knowledge: Ensure questions range from fundamental concepts to advanced problem-solving, covering both theory and application.
        Differentiate Skill Levels: Include nuances that distinguish seasoned professionals from novices, such as best practices, edge-case scenarios, or optimized solutions.
        Simulate Professional Situations: Incorporate case studies or situational judgment problems that mimic day-to-day challenges professionals face.
        Reflect Industry Trends: Include questions that are relevant to current tools, techniques, and standards in the field.
        Each question should have a brief context or scenario.
        Structure questions only on advanced levels of difficulty
        output your questions in valid json format where new line is strictly prohitbited. only produce it in compact format. Strictly follow this example Json Output Structure:[{"question":"[Insert question here]","options":["[Option 1]","[Option 2]","[Option 3]","[Option 4]"],"answer":"[a/b/c/d]","reason":"[Explanation for the correct answer.]"},{"question":"[Insert question here]","options":["[Option 1]","[Option 2]","[Option 3]","[Option 4]"],"answer":"[a/b/c/d]","reason":"[Explanation for the correct answer.]"}]
        only output the required json. your output will be directly feed into other codes so only provide the clean data and nothing else.""",
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db"),
        instructions=["you only output what is reuqired and never provide explanation or additional information."],
        add_history_to_messages=True,
        num_history_responses=3,
        
    )
    return mc_agent.run(prompt.format(role=role, description=description)).content, mc_agent.session_id